In [ ]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import*

spark = SparkSession.builder.appName("abs").getOrCreate()

# df = spark.read.csv("./data/part-10/2019-10-01.csv",header=True, inferSchema=True)
# df.count()
#ngay 1 có 1006239 dòng
df = spark.read.csv("./kaggle-data/2019-10.csv",header=True, inferSchema=True)
# df1 = df.where(col("event_type")=="purchase").groupBy("user_id").agg(sum("price").alias("total_spend"))

+--------------------+---------+----------------+-----------------+-----------------+------------------+
|          event_time|  user_id|interaction_type|   issue_category|resolution_status|satisfaction_score|
+--------------------+---------+----------------+-----------------+-----------------+------------------+
|2019-11-27T04:39:...|522700692|            call|  product_inquiry|         resolved|                 2|
|2019-10-14T18:06:...|524795442|          ticket| billing_question|         resolved|                 3|
|2019-10-07T11:08:...|548885191|            call| billing_question|         resolved|                 1|
|2019-10-10T13:06:...|541414887|            chat|   delivery_issue|         resolved|                 3|
|2019-11-29T20:07:...|561644129|            chat| billing_question|         resolved|                 1|
|2019-11-22T05:18:...|554284538|            call|technical_support|         resolved|                 1|
|2019-11-14T04:40:...|550091484|            chat| billi

In [12]:
# df1.orderBy(col("total_spend").desc()).show()
df2 = df.groupBy("user_id").agg(count_distinct("user_session").alias("total_visit"))
df2.orderBy(col("total_visit").desc()).show()

+---------+-----------+
|  user_id|total_visit|
+---------+-----------+
|512475445|       7400|
|526731152|       2773|
|514649263|       2336|
|563459593|       1618|
|561163588|       1182|
|554631275|        860|
|512500203|        785|
|513847031|        731|
|548931675|        681|
|537873067|        673|
|556593461|        669|
|553292256|        587|
|512365995|        406|
|515794073|        347|
|512386032|        328|
|529839452|        327|
|538019949|        314|
|512443087|        313|
|512661902|        299|
|538593075|        286|
+---------+-----------+
only showing top 20 rows



In [ ]:
t1 = df.where(to_date("event_time")=="2019-10-02").dropDuplicates().count()
# t2 = df.dropDuplicates().count()

#file goc ngay thu 2 xoa trung co 1 220 120 dong, trong duckdb 1 191 328 dong

In [20]:
t1

1220120

In [73]:
# lịch sử mua hàng
df1 = df.filter(col("event_type") == "purchase")\
.groupBy("user_id", "user_session", "product_id")\
.agg(
    count("*").alias("quantity"),
    first("price").alias("price")  # hoặc avg("price") nếu giá có thể thay đổi
)
order_time_df_1 = df.filter(col("event_type") == "purchase")\
        .groupBy("user_id", "user_session")\
        .agg(min("event_time").alias("order_timestamp"))

df2 = df1.groupBy("user_id", "user_session").agg(
    sum(col("quantity") * col("price")).alias("total_amount"),
    collect_list(struct(
        col("product_id"),
        col("quantity"),
        col("price")
    )).alias("items")
)
purchase_history_1 = df2.join(order_time_df_1, on=["user_id", "user_session"])
purchase_history_1= purchase_history_1.withColumnRenamed("user_session","order_id")
# purchase_history_1.where(col('user_id') == "552265052").show(truncate=False)

#ngay thu 2
df_2 = spark.read.csv("./data/part-10/2019-10-04.csv", header=True, inferSchema=True)
df_2 = df_2.withColumn("event_time", to_timestamp("event_time"))

# Lọc purchase và tính quantity, price
df3 = df_2.filter(col("event_type") == "purchase") \
    .groupBy("user_id", "user_session", "product_id") \
    .agg(
        count("*").alias("quantity"),
        first("price").alias("price")  # hoặc avg("price")
    )

# Tính thời gian order đầu tiên
order_time_df_2 = df_2.filter(col("event_type") == "purchase") \
    .groupBy("user_id", "user_session") \
    .agg(min("event_time").alias("order_timestamp"))

# Gom đơn
df4 = df3.groupBy("user_id", "user_session").agg(
    sum(col("quantity") * col("price")).alias("total_amount"),
    collect_list(struct(
        col("product_id"),
        col("quantity"),
        col("price")
    )).alias("items")
)

# Join thời gian order vào
purchase_history_2 = df4.join(order_time_df_2, on=["user_id", "user_session"])
purchase_history_2 = purchase_history_2.withColumnRenamed("user_session", "order_id")
purchase_history_2.where(col('user_id') == "552265052").show(truncate = False)
purchase_history = purchase_history_1.join(purchase_history_2.withColumnRenamed("user_session", "order_id"), "user_id", "left")
purchase_history_2=purchase_history_2.groupBy("user_id").agg(
    collect_list(struct(
        "order_id",
        "order_timestamp",
        "total_amount",
        "items" \
    )).alias("purchase_history")
)
purchase_history_2.where(col('user_id') == "552265052").show(truncate = False)
t = purchase_history_2\
    .where(col('user_id') == "552265052").select("user_id",explode("purchase_history").alias("order"))
t.select('order.*').show()

+---------+------------------------------------+------------+--------------------------------------------+-------------------+
|user_id  |order_id                            |total_amount|items                                       |order_timestamp    |
+---------+------------------------------------+------------+--------------------------------------------+-------------------+
|552265052|788c91d9-636d-4c42-8647-24133c8f7cf4|696.77      |[{6700993, 1, 346.98}, {6701183, 1, 349.79}]|2019-10-04 11:05:19|
+---------+------------------------------------+------------+--------------------------------------------+-------------------+

+---------+-------------------------------------------------------------------------------------------------------------------+
|user_id  |purchase_history                                                                                                   |
+---------+-------------------------------------------------------------------------------------------------

In [51]:
#mua gan day
recent_df = purchase_history.select(
    "user_id", "order_timestamp", explode("items").alias("item")
).select(
    "user_id",
    "order_timestamp",
    col("item.product_id").alias("product_id")
)
window = Window.partitionBy("user_id").orderBy(col("order_timestamp").desc())
recent_df = recent_df.withColumn("rank", row_number().over(window))
recent_df.filter(col("rank")<=5)\
.groupBy("user_id").agg(collect_list("product_id").alias("recent_purchase")).show(4)

+---------+---------------+
|  user_id|recent_purchase|
+---------+---------------+
|440756116|     [31500231]|
|451971397|      [2800203]|
|453705583|     [22600031]|
|457360398|      [6200373]|
+---------+---------------+
only showing top 4 rows



In [84]:
# the loai yeu thich StructField("category_preferences", MapType(StringType(), FloatType()), True),

event_weights={"view": 0.1, "cart": 0.2, "purchase":0.3}
scored_events = df.withColumn(
        "score",
        when(col("event_type") == "view", lit(event_weights.get("view", 0)))
        .when(col("event_type") == "purchase", lit(event_weights.get("purchase", 0)))
        .when(col("event_type") == "cart", lit(event_weights.get("cart", 0)))
        .otherwise(lit(0.0)))\
                .withColumn("category_code", coalesce("category_code", lit("unknow")))
category_preferences = scored_events.groupBy("user_id","category_code").agg(sum("score").alias("total_score"))\
.withColumn("total_score",round(col("total_score"),2))\
.groupBy("user_id").agg(map_from_entries((collect_list(struct("category_code","total_score")))).alias("category_preferences"))
category_preferences.show(truncate=False)

+---------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|user_id  |category_preferences                                                                                                                                  |
+---------+------------------------------------------------------------------------------------------------------------------------------------------------------+
|244951053|{electronics.smartphone -> 0.2}                                                                                                                       |
|330585300|{appliances.environment.vacuum -> 0.6}                                                                                                                |
|351866718|{furniture.kitchen.table -> 0.1}                                                                                                                      |
|388541724|{kids.toys 

In [13]:
snapshot_date = "2019-11-20"
df_churn = df.groupBy("user_id").agg(
        min(col("event_time")).alias("first_activity_date"),
        max(when(col("event_type") == "purchase", col("event_time"))).alias("last_purchase_date"),
        max(when(col("event_type").isin("view","cart"), col("event_time"))).alias("last_active_day")
    )

# df_churn = df_churn.withColumn("days_since_last_purchase", datediff(lit(snapshot_date), col("last_purchase_date")))\
#                        .withColumn("days_since_last_activity", datediff(lit(snapshot_date), col("last_active_day")))

# df_churn = df_churn.withColumn("churn_risk",
#         when(col("days_since_last_purchase") > 30, "High")
#         .when(col("days_since_last_activity") > 51, "High")
#         .when((col("days_since_last_purchase") > 15) & (col("days_since_last_activity") < 30), "Medium")
#         .otherwise("Low"))
df_churn.show(5)

+---------+-------------------+------------------+-------------------+
|  user_id|first_activity_date|last_purchase_date|    last_active_day|
+---------+-------------------+------------------+-------------------+
|555447570|2019-10-01 07:00:28|              NULL|2019-10-01 07:09:55|
|524493281|2019-10-01 09:17:22|              NULL|2019-10-01 09:22:13|
|543091275|2019-10-01 09:17:38|              NULL|2019-10-01 09:24:07|
|509481306|2019-10-01 09:19:51|              NULL|2019-10-01 09:19:51|
|513992906|2019-10-01 09:20:45|              NULL|2019-10-01 09:21:27|
+---------+-------------------+------------------+-------------------+
only showing top 5 rows



In [29]:
time_stamp = df.groupBy("user_id").agg(
           min("event_time").alias("first_visit_timestamp"),
           max("event_time").alias("last_visit_timestamp"),
           max(when(col("event_type")=="purchase",col("event_time"))).alias("last_purchase_date"),
           max(when(col("event_type").isin("view","cart"), col("event_time"))).alias("last_active_day"),
           countDistinct("user_session").alias("total_visits"))
time_stamp.select("user_id", "first_visit_timestamp","last_visit_timestamp","last_purchase_date","last_active_day","total_visits").show(5)



+---------+---------------------+--------------------+------------------+-------------------+------------+
|  user_id|first_visit_timestamp|last_visit_timestamp|last_purchase_date|    last_active_day|total_visits|
+---------+---------------------+--------------------+------------------+-------------------+------------+
|513992906|  2019-10-01 09:20:45| 2019-10-01 09:21:27|              NULL|2019-10-01 09:21:27|           1|
|530039999|  2019-10-01 09:47:08| 2019-10-01 10:02:42|              NULL|2019-10-01 10:02:42|           2|
|521210344|  2019-10-01 10:49:33| 2019-10-01 15:25:20|              NULL|2019-10-01 15:25:20|           4|
|553649979|  2019-10-01 09:51:05| 2019-10-01 09:55:02|              NULL|2019-10-01 09:55:02|           1|
|534133224|  2019-10-01 10:24:48| 2019-10-01 14:04:48|              NULL|2019-10-01 14:04:48|           3|
+---------+---------------------+--------------------+------------------+-------------------+------------+
only showing top 5 rows



In [28]:
visit = df.groupBy("user_id").agg(countDistinct("user_session").alias("total_Visit")).show(2)

+---------+-----------+
|  user_id|total_Visit|
+---------+-----------+
|513992906|          1|
|530039999|          2|
+---------+-----------+
only showing top 2 rows



In [27]:
df.filter(col("user_id")== "524493281").show()

+-------------------+----------+----------+-------------------+--------------------+-----+------+---------+--------------------+
|         event_time|event_type|product_id|        category_id|       category_code|brand| price|  user_id|        user_session|
+-------------------+----------+----------+-------------------+--------------------+-----+------+---------+--------------------+
|2019-10-01 09:17:22|      view|   1307073|2053013558920217191|  computers.notebook| acer| 669.0|524493281|bc8761db-6f7a-4da...|
|2019-10-01 09:18:03|      view|  26402619|2053013563651392361|                NULL| NULL| 382.1|524493281|bc8761db-6f7a-4da...|
|2019-10-01 09:18:58|      view|   3701359|2053013565983425517|appliances.enviro...|bosch|181.34|524493281|bc8761db-6f7a-4da...|
|2019-10-01 09:22:13|      view|   2800203|2053013563835941749|appliances.kitche...| NULL|199.17|524493281|bc8761db-6f7a-4da...|
+-------------------+----------+----------+-------------------+--------------------+-----+------+

In [90]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import*
from pyspark.sql.window import Window
from pyspark.sql.types import *
from datetime import datetime, timedelta
user_profile_schema = StructType([
    StructField("user_id", StringType(), False),
    StructField("first_visit_timestamp", TimestampType(), True),
    StructField("last_visit_timestamp", TimestampType(), True),
    StructField("last_purchase_date", TimestampType(), True),
    StructField("last_active_date", TimestampType(), True),
    StructField("total_visits", IntegerType(), True),
    StructField("purchase_history", ArrayType(StructType([
        StructField("order_id", StringType(), False),
        StructField("order_timestamp", TimestampType(), False),
        StructField("total_amount", FloatType(), False),
        StructField("items", ArrayType(StructType([
            StructField("product_id", StringType(), False),
            StructField("quantity", IntegerType(), False),
            StructField("price", FloatType(), False)
        ])), False)
    ])), True),
    StructField("recent_purchases", ArrayType(StringType()), True),
    StructField("category_preferences", MapType(StringType(), FloatType()), True),
    StructField("brand_preferences", MapType(StringType(), FloatType()), True),
    StructField("churn_prediction", StructType([
        StructField("probability", FloatType(), True),
        StructField("predicted_class", BooleanType(), True),
        StructField("last_prediction_timestamp", TimestampType(), True)
    ]), True),
    StructField("recommendations", StructType([
        StructField("personalized", ArrayType(StringType()), True),
        StructField("trending", ArrayType(StringType()), True)
    ]), True),
    StructField("segments", StringType(), True),
    StructField("last_used_timestamp", TimestampType(), True)
])

def compute_timestamp(df_logs):
    #Tính toán thời điểm người dùng xuất hiện lần đầu tiên và lần cuối dựa trên log sự kiện.
    return df_logs.groupBy("user_id").agg(
           min("event_time").alias("first_visit_timestamp"),
           max("event_time").alias("last_visit_timestamp"),
           max(when(col("event_type")=="purchase",col("event_time"))).alias("last_purchase_date"),
           max(when(col("event_type").isin("view","cart"), col("event_time"))).alias("last_active_date"),
           countDistinct("user_session").alias("total_visits"))\
            .select("user_id", "first_visit_timestamp","last_visit_timestamp","last_purchase_date","last_active_date","total_visits")
 
def compute_purchase_history(df_logs):
       product_agg = df_logs.filter(col("event_type") == "purchase")\
        .groupBy("user_id", "user_session", "product_id")\
        .agg(
            count("*").alias("quantity"),
            first("price").alias("price")
            )
       order_time_df = df_logs.filter(col("event_type") == "purchase")\
        .groupBy("user_id", "user_session")\
        .agg(min("event_time").alias("order_timestamp"))
       
       purchase_history_df = product_agg.groupBy("user_id", "user_session").agg(
        sum(col("quantity") * col("price")).alias("total_amount"),
        collect_list(
            struct(col("product_id"), col("quantity"), col("price"))
            ).alias("items"))
       
       purchase_history = purchase_history_df.join(order_time_df, on =["user_id","user_session"])\
                                            .withColumnRenamed("user_session","order_id")
       daily_purchase_history = purchase_history.groupBy("user_id").agg(
        collect_list(
            struct(col("order_id"),col("order_timestamp"),col("total_amount"),col("items"))
        ).alias("purchase_history")
    )
       return daily_purchase_history

def compute_recent_purchase(purchase_history):
        purchase_history = purchase_history.withColumn("order",explode(col("purchase_history")))
        purchase_history = purchase_history.select("user_id","order.*")
        recent_df = purchase_history.select(
            "user_id", "order_timestamp", explode("items").alias("item")
        ).select("user_id", "order_timestamp", col("item.product_id").alias("product_id"))
        
        window = Window.partitionBy("user_id").orderBy(col("order_timestamp").desc())
        recent_df = recent_df.withColumn("rank", row_number().over(window))
        recent_purchase = recent_df.filter(col("rank")<=5)\
        .groupBy("user_id").agg(collect_list("product_id").alias("recent_purchase"))
        return recent_purchase

def compute_category_preferences(df_logs):
        event_weights={"view": 0.1, "cart": 0.2, "purchase":0.3}
        scored_events = df_logs.withColumn("score",
                    when(col("event_type") == "view", lit(event_weights.get("view", 0)))
                    .when(col("event_type") == "purchase", lit(event_weights.get("purchase", 0)))
                    .when(col("event_type") == "cart", lit(event_weights.get("cart", 0)))
                    .otherwise(lit(0.0)))\
                                .withColumn("category_code", coalesce("category_code", lit("unknow")))
        
        category_preferences = scored_events.groupBy("user_id","category_code").agg(sum("score").alias("total_score"))\
        .withColumn("total_score",round(col("total_score"),2))\
        .groupBy("user_id").agg(map_from_entries((collect_list(struct("category_code","total_score")))).alias("category_preferences"))

        return category_preferences

def compute_brand_preferences(df_logs):
        event_weights={"view": 0.1, "cart": 0.2, "purchase":0.3}
        scored_events = df_logs.withColumn("score",
                    when(col("event_type") == "view", lit(event_weights.get("view", 0)))
                    .when(col("event_type") == "purchase", lit(event_weights.get("purchase", 0)))
                    .when(col("event_type") == "cart", lit(event_weights.get("cart", 0)))
                    .otherwise(lit(0.0)))\
                                .withColumn("brand", coalesce("brand", lit("unknow")))
        
        brand_preferences = scored_events.groupBy("user_id","brand").agg(sum("score").alias("total_score"))\
        .withColumn("total_score",round(col("total_score"),2))\
        .groupBy("user_id").agg(map_from_entries((collect_list(struct("brand","total_score")))).alias("brand_preferences"))
        return brand_preferences


def pro():
        spark = SparkSession.builder \
        .appName("Transformation") \
        .master("spark://spark-master:7077")\
        .getOrCreate()

        df_logs = spark.read.csv("./data/part-10/2019-10-01.csv",header=True, inferSchema=True)

        timestamp = compute_timestamp(df_logs)
        purchase_history = compute_purchase_history(df_logs)
        recent_purchase = compute_recent_purchase(purchase_history)
        category_preferences = compute_category_preferences(df_logs)
        brand_preferences = compute_brand_preferences(df_logs)

        df_profile = spark.createDataFrame([], schema=user_profile_schema)

        # Kết hợp các thành phần mới với profile hiện tại
        profile_update = df_profile.join(timestamp, "user_id", "outer")\
                        .withColumn("first_visit_timestamp",least(df_profile["first_visit_timestamp"], timestamp["first_visit_timestamp"]))\
                        .withColumn("last_visit_timestamp",greatest(df_profile["last_visit_timestamp"], timestamp["last_visit_timestamp"]))\
                        .withColumn("last_purchase_date",
                                    greatest(
                                        coalesce(df_profile["last_purchase_date"],lit(None)), 
                                        coalesce(timestamp["last_purchase_date"],lit(None))
                                        )
                                    )\
                        .withColumn("last_active_date",
                                    greatest(
                                        coalesce(df_profile["last_active_date"],lit(None)), 
                                        coalesce(timestamp["last_active_date"]),lit(None)
                                        )
                                    )\
                        .withColumn("total_visits",coalesce(df_profile["total_visits"],lit(0)) + coalesce(timestamp["total_visits"],lit(0)))
        return profile_update
a = pro()
a.printSchema()

root
 |-- user_id: string (nullable = true)
 |-- first_visit_timestamp: timestamp (nullable = true)
 |-- last_visit_timestamp: timestamp (nullable = true)
 |-- last_purchase_date: timestamp (nullable = true)
 |-- last_active_date: timestamp (nullable = true)
 |-- total_visits: long (nullable = false)
 |-- purchase_history: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- order_id: string (nullable = false)
 |    |    |-- order_timestamp: timestamp (nullable = false)
 |    |    |-- total_amount: float (nullable = false)
 |    |    |-- items: array (nullable = false)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- product_id: string (nullable = false)
 |    |    |    |    |-- quantity: integer (nullable = false)
 |    |    |    |    |-- price: float (nullable = false)
 |-- recent_purchases: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- category_preferences: map (nullable = true)
 |    

In [1]:
import duckdb
con = duckdb.connect("./data/2019-10.duckdb")
# con.execute(f"CREATE TABLE IF NOT EXISTS logs AS SELECT * FROM read_csv('./kaggle-data/2019-10.csv')")
# con.execute("CREATE SEQUENCE serial START 1")
# con.execute("ALTER TABLE logs ADD COLUMN id BIGINT DEFAULT nextval('serial')")
# con.execute("CHECKPOINT")




In [2]:
# #ngay 1: 1244245 dong
query1 ="""

"""
query = """
CREATE TABLE logs_test AS
SELECT 
    row_number() OVER () AS id,
    *
FROM read_csv_auto('./kaggle-data/2019-10.csv');
"""

# t=con.execute(query).fetchall()
# print(t)
# query2="""
# SELECT event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session, COUNT(*) as cnt
# FROM logs
# WHERE cast(event_time as date) == '2019-10-01'
# GROUP BY event_time, event_type, product_id, category_id, category_code, brand, price, user_id, user_session
# HAVING COUNT(*) > 1
# ORDER BY cnt DESC"""
con.execute(query).fetchall()
# print(t)
# con.execute("DROP TABLE IF EXISTS logs_with_id")
tables = con.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'main'").fetchall()

# Hiển thị kết quả
for table in tables:
    print(table[0])

logs
logs_test


In [11]:
con.execute("select COUNT(*) AS total_records, count(distinct user_id) from logs where cast(event_time as date) == '2019-10-02' ").fetchall()

[(1191328, 184965)]

In [55]:
from pyspark.sql import SparkSession    
from pyspark.sql.functions import *
spark = SparkSession.builder.appName("RECO ALS").getOrCreate()
testdf = spark.read.csv("./data/part-10/2019-10-02.csv",header=True)

In [56]:
testdf = testdf.where(col("user_id")=="525454715")\
                .where(col("event_type")=="purchase")
testdf.show()

+--------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|          event_time|event_type|product_id|        category_id|       category_code|  brand| price|  user_id|        user_session|
+--------------------+----------+----------+-------------------+--------------------+-------+------+---------+--------------------+
|2019-10-02 03:46:...|  purchase|   1004833|2053013555631882655|electronics.smart...|samsung|174.34|525454715|fffc6f01-ba1f-4f9...|
|2019-10-02 03:47:...|  purchase|   1004833|2053013555631882655|electronics.smart...|samsung|174.34|525454715|fffc6f01-ba1f-4f9...|
|2019-10-02 04:54:...|  purchase|   1004833|2053013555631882655|electronics.smart...|samsung|174.34|525454715|f721c3d4-42ee-4a4...|
|2019-10-02 05:00:...|  purchase|   1004833|2053013555631882655|electronics.smart...|samsung|174.34|525454715|f721c3d4-42ee-4a4...|
|2019-10-02 05:00:...|  purchase|   1004833|2053013555631882655|electronics.

In [62]:
def compute_purchase_history(df_logs):
    product_agg = df_logs.filter(col("event_type") == "purchase") \
        .groupBy("user_id", "user_session", "product_id") \
        .agg(
            count("*").alias("quantity"),
            round(first("price")).alias("price"),
            min("event_time").alias("order_timestamp")
        )

    purchase_history_df = product_agg.groupBy("user_id", "user_session").agg(
        round(sum(col("quantity") * col("price")), 2).alias("total_amount"),
        first("order_timestamp").alias("order_timestamp"),
        collect_list(
            struct(col("product_id"), col("quantity"), col("price"))
        ).alias("items")) \
        .withColumnRenamed("user_session", "order_id")

    daily_purchase_history = purchase_history_df.groupBy("user_id").agg(
        collect_list(
            struct(col("order_id"), col("order_timestamp"), col("total_amount"), col("items"))
        ).alias("purchase_history"))
    return daily_purchase_history
testd = compute_purchase_history(testdf)
t1= testd.select("user_id",explode("purchase_history").alias("order"))\
        .select("user_id","order.*")
t1.show()

+---------+--------------------+--------------------+------------+--------------------+
|  user_id|            order_id|     order_timestamp|total_amount|               items|
+---------+--------------------+--------------------+------------+--------------------+
|525454715|51e2a4e2-65ea-4d8...|2019-10-02 12:39:...|       174.0|[{1004833, 1, 174...|
|525454715|fffc6f01-ba1f-4f9...|2019-10-02 03:46:...|       348.0|[{1004833, 2, 174...|
|525454715|f54784a6-be60-4f3...|2019-10-02 10:00:...|       174.0|[{1004833, 1, 174...|
|525454715|f721c3d4-42ee-4a4...|2019-10-02 04:54:...|       522.0|[{1004833, 3, 174...|
+---------+--------------------+--------------------+------------+--------------------+



In [54]:
from pyspark.sql.window import Window

purchase_history = testd.withColumn("order",explode(col("purchase_history")))
purchase_history = purchase_history.select("user_id","order.*")
recent_df = purchase_history.select(
            "user_id", "order_timestamp", explode("items").alias("item")
        ).select("user_id", "order_timestamp", col("item.product_id").alias("product_id"),col("item.price").alias("price"),col("item.quantity").alias("quantity"))
window = Window.orderBy(col("order_timestamp").desc()).partitionBy("user_id")
recent_df=recent_df.withColumn("rank", row_number().over(window))\
            .groupBy("user_id","product_id").agg(sum("quantity").alias("quantity"))\
                                                                .groupBy("user_id").agg(collect_list(struct("product_id","quantity")).alias("recent_purchases"))

recent_df.show()

+---------+----------------+
|  user_id|recent_purchases|
+---------+----------------+
|525454715|  [{1004833, 7}]|
+---------+----------------+

